In [ ]:
# ======================================
#  Notebook: Face Detection + Recognition
# ======================================

# 1. Setup do ambiente
!pip install mtcnn keras-vggface keras-applications scikit-learn

import cv2
import matplotlib.pyplot as plt
import numpy as np
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from sklearn.svm import SVC

In [ ]:
# 2. Carregar detector de faces (MTCNN)
detector = MTCNN()

In [ ]:
# 3. Carregar modelo VGGFace para embeddings
model = VGGFace(model="resnet50", include_top=False, pooling="avg")

# Função para extrair rostos
def extract_faces(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(img_rgb)
    face_images = []
    boxes = []
    for f in faces:
        x, y, w, h = f['box']
        face = img_rgb[y:y+h, x:x+w]
        face_images.append(face)
        boxes.append((x, y, w, h))
    return img_rgb, face_images, boxes

# Função para gerar embedding
def get_embedding(face_pixels):
    face_pixels = cv2.resize(face_pixels, (224, 224))
    face_pixels = np.expand_dims(face_pixels, axis=0).astype('float32')
    face_pixels = preprocess_input(face_pixels, version=2)
    embedding = model.predict(face_pixels)
    return embedding[0]

In [ ]:
# 4. Exemplo de dataset (adicione imagens suas aqui)
# Exemplo fictício com 2 classes: pessoa1.jpg, pessoa2.jpg
X, y = [], []

for img_path, label in [("pessoa1.jpg", "Pessoa 1"), ("pessoa2.jpg", "Pessoa 2")]:
    img_rgb, faces, boxes = extract_faces(img_path)
    for face in faces:
        emb = get_embedding(face)
        X.append(emb)
        y.append(label)

X = np.array(X)
y = np.array(y)

In [ ]:
# 5. Treinar classificador SVM
clf = SVC(kernel='linear', probability=True)
clf.fit(X, y)

In [ ]:
# 6. Testar em nova imagem
img_rgb, faces, boxes = extract_faces("teste.jpg")

for i, face in enumerate(faces):
    emb = get_embedding(face)
    pred = clf.predict([emb])[0]
    x, y0, w, h = boxes[i]
    cv2.rectangle(img_rgb, (x,y0), (x+w,y0+h), (0,255,0), 2)
    cv2.putText(img_rgb, pred, (x, y0-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

plt.imshow(img_rgb)
plt.axis("off")
plt.show()